In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_ksf_grid_correction_parallel.ipynb
# AUTHOR - James Lane
# PROJECT - ges_mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Parallelize the creation of the KSF grid
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np, sys, os, copy, warnings, operator, time, multiprocessing
from astropy import units as apu
from matplotlib import pyplot as plt
import matplotlib as mpl
import dill as pickle
import tqdm

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA
from galpy import df
from galpy.util import multi as galpy_multi

## scipy
from scipy import interpolate

## Project-specific
sys.path.insert(0,'../../src/')
from ges_mass import potential as ppotential
from ges_mass import plot as pplot
from ges_mass import util as putil
from ges_mass import ssf as pssf

In [ ]:
# Notebook setup
%matplotlib inline
plt.style.use('jl_notebook') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
ncores = int(multiprocessing.cpu_count()//2) # Include in 

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','RO','VO','ZO']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ro,vo,zo\
    = putil.parse_config_dict(cdict,keywords)

# Pathing
data_dir = data_dir_base+'gaia_apogee/'
out_dir = data_dir_base+'ksf/'
fig_dir = './fig/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
os.makedirs(out_dir+gaia_apogee_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)

# Filenames
df_filename = out_dir+gaia_apogee_dir+'dfs.pkl'
allstar_filename = data_dir+gaia_apogee_dir+'apogee_allstar.npy'
apogee_stat_indx_filename = data_dir+gaia_apogee_dir+'apogee_statIndx.npy'
gaia_data_filename = data_dir+gaia_apogee_dir+'gaia_data.npy'
gaia_apogee_matches_filename = data_dir+gaia_apogee_dir+'gaia_apogee_matches.npy'
kinematics_filename = out_dir+gaia_apogee_dir+'kinematics_ksf_correction.pkl'

# Forcing
force_dfs = False # New DFs
force_kinematics = False # New orbit samples
force_splines = False # New splines of completeness and purity

## Make potential and DFs

In [ ]:
# Interpolated Milky Way potential
rmin = 1/ro # 1 kpc
rmax = 60/ro # 60 kpc
rmin_interp = rmin/2.
rmax_interp = rmax*2.
ngrid = 1601
interpot = ppotential.make_interpolated_mwpot(mwpot='MWPotential2014',
    rmin=rmin_interp, rmax=rmax_interp, ngrid=ngrid, ro=ro, vo=vo, 
    match_type='mass')
mwpot = potential.MWPotential2014
potential.turn_physical_on(interpot,ro=ro,vo=vo)
potential.turn_physical_on(mwpot,ro=ro,vo=vo)
phi0 = potential.evaluatePotentials(mwpot,1e12,0).value

# Stellar halo density potential
alpha = 3.5 # halo density inner power law slope
rc = 30*apu.kpc
denspot = potential.PowerSphericalPotentialwCutoff(amp=1., r1=1.,
    alpha=alpha, rc=rc, ro=ro, vo=vo)
potential.turn_physical_on(denspot,ro=ro,vo=vo)

In [ ]:
# Make DFs
betas = [0.5,0.9]
n_dfs = len(betas)
n_samples = 800 # samples in each distance modulus bin

if force_dfs or not os.path.exists(df_filename):
    dfs = []
    for i in range(len(betas)):
            # DF initialization is noisy
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                dfm = df.constantbetadf(pot=interpot, denspot=denspot, ro=ro, vo=vo, 
                    beta=betas[i], rmax=rmax)
                # Dummy samping to initialize
                _ = dfm.sample(R=np.ones(n_samples)*ro*apu.kpc, 
                    phi=np.zeros(n_samples), z=np.zeros(n_samples), rmin=rmin)
                dfs.append(dfm)
            ##wi
        ###i
    with open(df_filename,'wb') as f:
        pickle.dump(dfs,f)
    ##wi
else:
    with open(df_filename,'rb') as f:
        print('Loading DFs from '+df_filename)
        dfs = pickle.load(f)
    ##wi
    check_params = ['_beta','_rmin_sampling','_rmax','_pot','_denspot','_denspot.alpha','_denspot.rc']
    for i in range(len(dfs)):
        print('\ndf['+str(i)+'] properties')
        for j in range(len(check_params)):
            print(check_params[j]+': '+str(operator.attrgetter(check_params[j])(dfs[i])))
        ###j
    ###i
##ie

## Gaia and APOGEE data

In [ ]:
# Load APOGEE data
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
print('Loading APOGEE from '+allstar_filename)
allstar = np.load(allstar_filename)
print(str(len(allstar))+' stars in total sample.')

# load APOGEE statistical sample index
print('\nLoading APOGEE DR16 statistical sample from '+apogee_stat_indx_filename)
apogee_stat_indx = np.load(apogee_stat_indx_filename)
print(str(np.sum(apogee_stat_indx))+' stars in statistical sample.')

# Gaia data and Gaia-APOGEE match index
print('\nGaia data release is: '+gaia_dr)
print('Loading Gaia catalog from '+gaia_data_filename)
gaia_data = np.load(gaia_data_filename, allow_pickle=True)
print('Loading Gaia-APOGEE matches from '+gaia_apogee_matches_filename)
gaia_apogee_matches_indx = np.load(gaia_apogee_matches_filename)

# Apply the statistical sample index and Gaia-APOGEE matching index
allstar_gaia = allstar[apogee_stat_indx][gaia_apogee_matches_indx]

In [ ]:
# Should have defined 6D kinematics for eccentricities
input_mask = np.isfinite(gaia_data['RA']) &\
             np.isfinite(gaia_data['DEC']) &\
             np.isfinite(gaia_data['pmra']) &\
             np.isfinite(gaia_data['pmdec']) &\
             np.isfinite(allstar_gaia['weighted_dist']) &\
             np.isfinite(allstar_gaia['VHELIO_AVG'])

allstar_input = allstar_gaia[input_mask]
gaia_input = gaia_data[input_mask]

# Make coordinate array -> orbits
vxvv = np.array([gaia_input['RA'],
                 gaia_input['DEC'],
                 allstar_input['weighted_dist']/1000,
                 gaia_input['pmra'],
                 gaia_input['pmdec'],
                 allstar_input['VHELIO_AVG']
                 ]).T
orbs_gaia_apo = orbit.Orbit(vxvv=vxvv, radec=True, ro=ro, vo=vo, zo=zo)

# Trim the size of gaia_input and allstar_input by only keeping some fields
gaia_input,allstar_input = putil.trim_gaia_allstar_input(gaia_input,
                                                         allstar_input)

## Get the locations of the APOGEE SF

In [ ]:
# Get individual pointing information
ls_pointing = np.load(data_dir+gaia_apogee_dir+'apogee_field_glons.npy')
bs_pointing = np.load(data_dir+gaia_apogee_dir+'apogee_field_glats.npy')
locids_pointing = np.load(data_dir+gaia_apogee_dir+'apogee_field_location_ids.npy')
n_pointing = len(locids_pointing)

# Get individual distance information
n_ds = 14
dmod_lim = [7,19]
dmods = np.linspace(dmod_lim[0],dmod_lim[1],num=n_ds) # About 1 to 50 kpc
ds_individual = 10**(dmods/5-2)
print('Distances [kpc]:')
print(ds_individual)

# Tile this information to create a grid of pointings x distances
ds = np.tile(ds_individual,reps=len(ls_pointing)) # repeat array one after the other
bs = np.repeat(bs_pointing,repeats=len(ds_individual)) # repeat each element
ls = np.repeat(ls_pointing,repeats=len(ds_individual))
fs = np.repeat(locids_pointing,repeats=len(ds_individual))

In [ ]:
# Plot the locations of APOGEE pointings
fig = plt.figure()
ax = fig.add_subplot(111)
ls_plot = copy.deepcopy(ls_pointing)
ls_plot[ ls_plot>180  ] = ls_plot[ ls_plot>180 ]-360
pts = ax.scatter( ls_plot, bs_pointing, c='Black', s=4, zorder=2 )
ax.set_xlabel('$\ell$ [deg]')
ax.set_ylabel('$b$ [deg]')
ax.set_xlim(185,-185)
ax.set_ylim(-95,95)
bulge_patch = mpl.patches.Rectangle(xy=(-20,-20),width=40,height=40,
    edgecolor='Black',facecolor='None', zorder=3)
ax.add_artist(bulge_patch)
fig.show()

## Make kinematics

In [ ]:
# Address weird galpy error
ls[0:n_ds] = 90.001 
ls[1050] = 90.001
ls[1060] = 90.001

# Make orbits for each individual location in the pointing x distance grid
vxvvs = np.array([ls,bs,ds,np.zeros_like(ds),np.zeros_like(ds),np.zeros_like(ds)]).T
orbs_locs = orbit.Orbit(vxvvs,lb=True,ro=ro,vo=vo,zo=zo) # Orbits to do coordinate tranformation
n_locs = len(orbs_locs)

ls = np.repeat(ls_pointing,repeats=len(ds_individual)) # Undo fudge for galpy error

In [ ]:
# Find matches into the gaia and apogee orbits
indx,sep2d,dist3d = putil.find_orbit_nearest_neighbor(orbs_locs,orbs_gaia_apo,ro=ro, vo=vo)
print('Max 2D angular separation')
print(sep2d.max())
print('\nMax 3D distance')
print(dist3d.max())

In [ ]:
ncores_temp = 4

if not os.path.exists(kinematics_filename) or force_kinematics:
    aAS = aA.actionAngleStaeckel(pot=mwpot, delta=0.4, ro=ro, vo=vo, zo=zo, c=True)
    do_perturb_orbs = True

    # Calculate deltas only once for each location
    print('Calculating Staeckel deltas...')
    deltas = aA.estimateDeltaStaeckel(mwpot, orbs_locs.R(), 
                                      orbs_locs.z(), no_median=True)
    
    orbs = []
    eELzs = np.zeros((len(dfs),n_locs,3,n_samples))
    actions = np.zeros((len(dfs),n_locs,3,n_samples))

    # Timing
    t1 = time.time()
    for i in range(len(dfs)):
        print('Doing beta='+str(dfs[i]._beta))
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            results = calc_kinematics_parallel(ncores_temp, dfs[i], n_samples,
                orbs_locs, do_perturb_orbs,gaia_input[indx], 
                allstar_input[indx], deltas, aAS, ro, vo, zo)
        ##wi
        these_orbs,these_eELzs,these_actions = results.T
        orbs.append( list(these_orbs) )
        eELzs[i] = np.stack(these_eELzs)
        actions[i] = np.stack(these_actions)

    t2 = time.time()
    print('Took '+str(round(t2-t1))+'s')
    print('Saving KSF kinematics to '+kinematics_filename)
    with open(kinematics_filename,'wb') as f:
        pickle.dump([deltas,eELzs,actions,orbs],f)
    ##wi
else:
    print('Loading KSF kinematics from '+kinematics_filename)
    with open(kinematics_filename,'rb') as f:
        deltas,eELzs,actions,orbs = pickle.load(f)
    ##wi
##ie

## Calculate Purity and Completeness & Fit Splines

In [ ]:
# Selection ellipse dictionary
halo_selection_survey_dict = {'vRvT':   [ ['ellipse', [290,0], [110,35]], 
                                          ['ellipse', [-290,0], [110,35]] ],
                              'Toomre': [ ['ellipse', [0,300], [35,120]], ],
                              'ELz':    [ ['ellipse', [0,-1], [300,0.5]], ],
                              'JRLz':   [ ['ellipse', [0,45], [300,20]], ],
                              'eLz':    [ ['ellipse', [0,1], [500,0.025]], ],
                              'AD':     [ ['ellipse', [0,-1], [0.08,0.3]], ]
                              }

# Plotting keywords
make_spline_plots = False

### Loop over different spaces

In [ ]:
selec_spaces = [['AD',],
                ['eLz',],
                ['JRLz'],
                ['AD','eLz'],
                ['AD','JRLz'],
                ['eLz','JRLz'],
               ]
for i in range(len(selec_spaces)):
    pssf.make_completeness_purity_splines(selec_spaces[i], orbs, eELzs, actions, 
        halo_selection_survey_dict, phi0,
        [ls_pointing,bs_pointing,locids_pointing], ds_individual, fs, out_dir, 
        gaia_apogee_dir, fig_dir, force_splines=False, make_spline_plots='both',
        n_spline_plots=20)